In [87]:
import numpy as np
import pandas as pd
import feyn
import sklearn.model_selection
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import f_classif
from scipy.stats import pointbiserialr


In [88]:
#Load data set
data =  pd.read_csv('C:/Users/riyak/OneDrive/Documents/NASA_2022_Summer/QL Ready Data/QL_GLDS105_All.csv')
data

,Treatment,ENSMUSG00000001334_RNA-seq,ENSMUSG00000001473_RNA-seq,ENSMUSG00000001627_RNA-seq,ENSMUSG00000001656_RNA-seq,ENSMUSG00000001995_RNA-seq,ENSMUSG00000003123_RNA-seq,ENSMUSG00000003476_RNA-seq,ENSMUSG00000003949_RNA-seq,ENSMUSG00000004035_RNA-seq,...,ENSMUSG00000020044,ENSMUSG00000075595,ENSMUSG00000071266,ENSMUSG00000003070,ENSMUSG00000027215,ENSMUSG00000101643,ENSMUSG00000021750,ENSMUSG00000105704,ENSMUSG00000030972,ENSMUSG00000101265
0,1,12.292794,8.981894,11.722759,9.092271,10.339797,9.984216,11.031675,9.305469,8.190129,...,0.751136,0.687015,0.686246,0.648847,0.737622,0.719424,0.774782,0.704485,0.762217,0.878012
1,1,12.226921,9.347813,11.724852,8.958192,10.311079,10.021252,10.815969,9.317519,8.215440,...,0.785657,0.664506,0.654434,0.711409,0.734929,0.744275,0.783732,0.689898,0.697220,0.829730
2,1,12.187921,9.018841,11.753646,9.016898,10.480373,10.019954,10.939154,9.640016,8.167023,...,0.773000,0.702000,0.733000,0.631000,0.738000,0.788000,0.732000,0.698000,0.684000,0.808000
3,1,12.368990,9.304828,11.784966,9.210343,10.393444,9.938769,11.242344,9.314816,8.238845,...,0.760369,0.661080,0.676150,0.677735,0.746457,0.760369,0.818925,0.729962,0.744757,0.867003
4,1,12.106554,9.232013,11.935793,9.179887,10.291489,9.872053,11.037053,9.460391,8.195981,...,0.759006,0.698485,0.686971,0.706202,0.740915,0.680435,0.763736,0.670358,0.750181,0.796380
5,1,12.101980,9.010526,11.723386,9.157278,10.222705,9.818746,11.116102,9.362643,8.312832,...,0.788077,0.689031,0.669795,0.669095,0.732957,0.735584,0.771644,0.697223,0.724221,0.839947
6,0,11.629508,8.870774,11.122903,8.867227,9.987957,9.928777,10.764668,9.772666,7.749969,...,0.784688,0.683974,0.694868,0.669080,0.759370,0.740446,0.791435,0.707351,0.750910,0.821488
7,0,11.649911,8.730011,11.170864,8.867193,9.929074,10.404779,10.851126,9.778077,7.822212,...,0.780676,0.684655,0.692940,0.654194,0.766967,0.730640,0.787080,0.695575,0.731834,0.864943
8,0,11.794002,8.805102,11.140699,8.924892,9.976810,10.230466,10.852532,9.859700,7.795119,...,0.793000,0.701000,0.696000,0.676000,0.754000,0.750000,0.769000,0.696000,0.719000,0.834000
9,0,11.740962,8.715806,11.089417,8.673586,10.021581,10.374602,10.611409,9.709360,7.788821,...,0.753218,0.666953,0.645558,0.692877,0.699874,0.758465,0.761029,0.691235,0.729167,0.839506


In [89]:
#Record categorical data types in dataset
stypes = {}
target = "Treatment"
for f in data.columns:
    if data[f].dtype =='object':
        stypes[f] = 'c'

In [117]:
average_120 = []
average_12 = []
features = []
ql = feyn.QLattice()
for i in range(0,11):
    
    train = data.drop(data.iloc[[i], :].index[0])
    
    # Instantiate a QLattice
    feyn.QLattice(random_seed = i)
    
    # Setting semantic types
    stypes = stypes

    # Set number of epochs
    n_epochs = 20

    # Initialize the list of models
    models = []
    
    # Sample and fit
    for epoch in range(n_epochs):
    
        # Sample models (no data here yet)
        models += ql.sample_models(
            input_names=train.columns, 
            output_name=target, 
            kind='classification', 
            stypes=stypes,
            max_complexity=6
        )
    
        # Fit the models with train data
        models = feyn.fit_models(models, train, loss_function='squared_error')
    
        # Remove worst performing models
        models = feyn.prune_models(models)
        
        # Update QLattice with the models sorted by loss
        ql.update(models)
    
    # Find the 10 best models
    best_models = models[:10]
    
    for j in range(10):
        feyn.show_model(best_models[j], update_display=False)
        average_120.append(best_models[j].accuracy_score(data))
        features.extend(best_models[j].features)
    best_model = models[0]
    average_12.append(best_model.accuracy_score(data))
    


C:\Users\riyak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\feyn\_functions.py:60: RuntimeWarning: overflow encountered in exp
  "func": lambda params, v: 1 / (1 + np.exp(-(v*params["w"]+params["bias"])))


C:\Users\riyak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\feyn\_functions.py:60: RuntimeWarning: overflow encountered in exp
  "func": lambda params, v: 1 / (1 + np.exp(-(v*params["w"]+params["bias"])))


C:\Users\riyak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\feyn\_functions.py:60: RuntimeWarning: overflow encountered in exp
  "func": lambda params, v: 1 / (1 + np.exp(-(v*params["w"]+params["bias"])))


C:\Users\riyak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\feyn\_functions.py:60: RuntimeWarning: overflow encountered in exp
  "func": lambda params, v: 1 / (1 + np.exp(-(v*params["w"]+params["bias"])))


C:\Users\riyak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\feyn\_functions.py:60: RuntimeWarning: overflow encountered in exp
  "func": lambda params, v: 1 / (1 + np.exp(-(v*params["w"]+params["bias"])))


In [118]:
from statistics import mean
mean(average_120)


0.996969696969697

In [119]:
mean(average_12)

1.0

In [120]:
features

['ENSMUSG00000001334_RNA-seq',
 'ENSMUSG00000026384_RNA-seq',
 'ENSMUSG00000026504',
 'ENSMUSG00000042793',
 'ENSMUSG00000031637_RNA-seq',
 'Q99MR9',
 'ENSMUSG00000001441',
 'ENSMUSG00000073700_RNA-seq',
 'ENSMUSG00000014778_RNA-seq',
 'ENSMUSG00000026028_RNA-seq',
 'ENSMUSG00000024642_RNA-seq',
 'ENSMUSG00000014778_RNA-seq',
 'ENSMUSG00000108375',
 'ENSMUSG00000024642_RNA-seq',
 'ENSMUSG00000014778_RNA-seq',
 'ENSMUSG00000028437',
 'ENSMUSG00000026028_RNA-seq',
 'Q3U561',
 'ENSMUSG00000001441',
 'ENSMUSG00000031637_RNA-seq',
 'ENSMUSG00000001441',
 'ENSMUSG00000031637_RNA-seq',
 'ENSMUSG00002076509',
 'ENSMUSG00000097375_RNA-seq',
 'O35864',
 'ENSMUSG00000029231',
 'ENSMUSG00000062980_RNA-seq',
 'ENSMUSG00000026135_RNA-seq',
 'Q6PB66',
 'ENSMUSG00000014778_RNA-seq',
 'ENSMUSG00000046345',
 'ENSMUSG00000031637_RNA-seq',
 'Q99MR9',
 'ENSMUSG00000001441',
 'ENSMUSG00000031637_RNA-seq',
 'ENSMUSG00000031637_RNA-seq',
 'ENSMUSG00002076509',
 'Q99MR9',
 'ENSMUSG00000019883',
 'ENSMUSG000000

In [121]:
#finding unique features

array = np.array(features)
unique, counts = np.unique(array, return_counts=True)
result = np.column_stack((unique, counts))
print(result)
len(result)

[['B1AQF4' '1']
 ['ENSMUSG00000000439' '1']
 ['ENSMUSG00000001334_RNA-seq' '2']
 ['ENSMUSG00000001441' '4']
 ['ENSMUSG00000001910' '1']
 ['ENSMUSG00000005204' '4']
 ['ENSMUSG00000012350' '2']
 ['ENSMUSG00000014778_RNA-seq' '6']
 ['ENSMUSG00000015711_RNA-seq' '2']
 ['ENSMUSG00000018585' '1']
 ['ENSMUSG00000019883' '1']
 ['ENSMUSG00000020275_RNA-seq' '2']
 ['ENSMUSG00000020547_RNA-seq' '1']
 ['ENSMUSG00000022501' '1']
 ['ENSMUSG00000024642_RNA-seq' '14']
 ['ENSMUSG00000025511_RNA-seq' '12']
 ['ENSMUSG00000025735_RNA-seq' '4']
 ['ENSMUSG00000026028_RNA-seq' '20']
 ['ENSMUSG00000026135_RNA-seq' '1']
 ['ENSMUSG00000026384_RNA-seq' '1']
 ['ENSMUSG00000026504' '1']
 ['ENSMUSG00000026872' '1']
 ['ENSMUSG00000027933' '1']
 ['ENSMUSG00000027998' '1']
 ['ENSMUSG00000028035_RNA-seq' '3']
 ['ENSMUSG00000028437' '1']
 ['ENSMUSG00000028630_RNA-seq' '1']
 ['ENSMUSG00000028979' '1']
 ['ENSMUSG00000029231' '1']
 ['ENSMUSG00000029765_RNA-seq' '2']
 ['ENSMUSG00000031430' '1']
 ['ENSMUSG00000031513' '2']
 

100

In [126]:
models = ql.auto_run(
    data=train,
    output_name='Treatment',
    kind='classification'
)